#### This notebook attempts to evaluate the merit of using the Bank Login score to identify "good" / "bad" customers.

As the Bank Login score already accounts for a lot of factors (~6k attributes!) we will use the "IsFirstDefault" Parameter to judge its correlation with loan outcomes.

In [375]:
import pymysql
import query
import pandas as pd
import seaborn as sns
import sidetable

In [335]:
df = pd.read_csv("/home/vishal/ftp_files_csv/bt_attr.csv",usecols=["setting_app_id","blp_score"])

In [ ]:
df.head()

In [ ]:
scoredf=df.rename(columns={"setting_app_id": "LoanId", "blp_score": "score"})

### Query the DB in the date range '2019-01-01' - '2020-04-30'

In [338]:
dbquerydf = query.iloans("SELECT * FROM view_FCL_Loan WHERE OriginationDate between '2019-01-01' AND '2020-04-30'")

In [ ]:
dbquerydf[["LoanId","IsFirstDefault","OriginationDate"]].info()

In [340]:
dbdf=dbquerydf[["LoanId","IsFirstDefault"]]

In [ ]:
dbdf.head()

In [342]:
view_df = pd.merge(scoredf, dbdf, how ='left', on ='LoanId') 

In [ ]:
view_df.info()

In [ ]:
view_df.head()

### Get Lender Approved Loans Data

In [345]:
query_lender_approved='''
select LoanId, 
LoanPrincipal AS ApprovedLoanAmount,
LoanStatus AS LenderApproved 
from view_FCL_Loan_History
where LoanStatus = 'Lender Approved' 
and TimeAdded >= '2019-01-01'
AND TimeAdded <= '2020-04-30'
ORDER BY LoanId , TimeAdded DESC
'''

In [346]:
dblender_approved_df = query.iloans(query_lender_approved)

In [ ]:
dblender_approved_df.head()

### Bin Data by Deciles and Remap LenderApproved to 0's/1's

In [348]:
view2_df = pd.merge(view_df, dblender_approved_df, how ='left', on ='LoanId') 
view2_df["IsFirstDefault"]*=1
view2_df['Decile_rank'] = pd.qcut(view2_df['score'], 10,labels = False) 
view2_df['ApprovedLoanAmount'] = view2_df['ApprovedLoanAmount'].fillna(0)
view2_df["LenderApproved"] = view2_df["LenderApproved"].replace(to_replace ="Lender Approved", value =1) 
view2_df["LenderApproved"] = view2_df["LenderApproved"].fillna(0)

In [ ]:
view2_df.head()

In [ ]:
view2_df.describe()

In [ ]:
view2_df.info()

### Calculate Loans approved,First Defaults and Total Loans per Decile

In [352]:
dropnaview2 = view2_df.dropna()
sums1 = []
sums = []
counts = []
for x in range(0,10):
    sums.append(view2_df[view2_df["Decile_rank"]==x].sum()["LenderApproved"])
    counts.append(view2_df[view2_df["Decile_rank"]==x].count()["LenderApproved"])
    sums1.append(dropnaview2[dropnaview2["Decile_rank"]==x].sum()["IsFirstDefault"])

In [353]:
deciledf = pd.DataFrame(list(zip(range(0,10),sums,sums1,counts)), columns =['Decile','LenderApprovedCount','IsFirstDefaultCount','TotalLoansCount']) 

### Calculate Percentage of Defaulters per Decile

In [354]:
avg = []
for x in range(0,10):
    avg.append((deciledf['IsFirstDefaultCount'][x]/deciledf['TotalLoansCount'][x])*100)

In [355]:
deciledf["PercentDefault"]=avg

### Calculate Cumulative Fraction of Defaulters per Decile

In [356]:
templist = []
sum = 0
for x in range(0,10):
    sum += (deciledf["IsFirstDefaultCount"][x]/deciledf["IsFirstDefaultCount"].sum())*100
    templist.append(sum)

In [357]:
deciledf["CumulativePercent"]= templist

In [358]:
deciledf["LenderPercent"]=(deciledf["LenderApprovedCount"]/deciledf["TotalLoansCount"])*100

In [359]:
testdeciledf=sorted(set(pd.qcut(view2_df['score'], 10,retbins= True)[0]))

In [360]:
randomlist = [0,1,2,3,4,5,6,7,8,9]

In [361]:
testdfdecile = pd.DataFrame(data = randomlist, columns = ["Decile"])

In [362]:
testdfdecile["Bins"] = testdeciledf

## Preparing the final table

In [363]:
finaldf = pd.merge(deciledf, testdfdecile, how ='left', on ='Decile') 

In [364]:
cols = list(finaldf.columns.values)

In [365]:
cols = ['Bins','Decile','TotalLoansCount','LenderApprovedCount','LenderPercent','IsFirstDefaultCount','PercentDefault','CumulativePercent']

In [366]:
finaldf=finaldf.reindex(columns=cols)
finaldf.columns = ['Score Range', 'Decile','NumberOfApps',"AppsApproved","FractionAppsApproved","AppsFirstDefault","FractionDefault","CumulativeDefaultFraction"]

In [368]:
finaldf.round(1)

,Score Range,Decile,NumberOfApps,AppsApproved,FractionAppsApproved,AppsFirstDefault,FractionDefault,CumulativeDefaultFraction
0,"(107.999, 260.0]",0,5747,2381.0,41.4,1656.0,28.8,72.2
1,"(260.0, 278.0]",1,6147,983.0,16.0,47.0,0.8,74.2
2,"(278.0, 284.0]",2,5333,1101.0,20.6,43.0,0.8,76.1
3,"(284.0, 288.0]",3,6127,1684.0,27.5,35.0,0.6,77.6
4,"(288.0, 291.0]",4,6744,2167.0,32.1,89.0,1.3,81.5
5,"(291.0, 293.0]",5,6216,2296.0,36.9,112.0,1.8,86.4
6,"(293.0, 294.0]",6,3663,1460.0,39.9,77.0,2.1,89.8
7,"(294.0, 296.0]",7,8404,3533.0,42.0,140.0,1.7,95.9
8,"(296.0, 297.0]",8,4124,1749.0,42.4,69.0,1.7,98.9
9,"(297.0, 299.0]",9,4062,1889.0,46.5,26.0,0.6,100.0


## Key Takeaways

1. Loan Applicants with a BL score of <260 are much more likely to default on their first payment.(72.2% of defaults across all loans!) 

2. Conversely, Applicants with a score of >260 are quite unlikely to default on their first payment.

Keeping the above two points in mind, we can safely say that the Bank Login score and the quality of application are <b>strongly correlated.</b>. 

Another interesting thing to note:

3. 89.1% of the Loan Applicants have a BL score of above 260 which account for only 27.8% of all first payment defaults.

### Opportunities Presented

1. A lot of applications with a BL score above 260 are not funded as often as the ones below 260. Maximising funding loans in this bracket will improve the first payment default rates.

2. Integrating the BL score should expedite the approvals process, improving the amount of loans funded in the higher deciles.

### Caveats

1. Due to the distribution of loans skewing heavily towards a BL score of 260+, a new question arises, "How many of these were in the A1/A2 categories in the BV+ score?"

It could be the case that most of these loans could fall into the A1/A2 Categories, and BL score wouldn't apply at all. In that case, we'll need to rethink how to use this score.